# Импорт библиотек

In [1]:
import numpy as np       # линейная алгебра
import pandas as pd      # работа с табличными данными
import os                # работа с операционной системой
import string            # работа со строковыми значениями
import re                # работа с регулярными выражениями

import nltk                               # работа с естественным языком           
from nltk.corpus import stopwords         # список стоп-слов

from sklearn.feature_extraction.text import TfidfVectorizer          # Преобразование тектов в TFIDF-матрицу
from sklearn.metrics.pairwise import cosine_similarity               # Метрика измерения косинусного расстояния между векторами

import gensim                                        # Работа с моделями W2V
import gensim.downloader
from gensim.models import word2vec,KeyedVectors

# Функции

In [2]:
# Функция удаления символов из текста
def remove_chars_from_text(text_, chars_):
    return "".join([ch for ch in text_ if ch not in chars_])


# Функция удаления стоп-слов
def clear_stop_words(text_, stopwords_):
    return ' '.join([word for word in str(text_).split() if word not in stopwords_])


# Функция удаления HTML-ссылок
def remove_urls(text_):
    url_remove = re.compile(r'https?://\S+|www\.\S+')
    return url_remove.sub(r' ', text_)


# Функция удаления URL-ссылок
def remove_html(text_):
    html=re.compile(r'<.*?>')
    return html.sub(r' ',text_)


# Функция предобработки текста
def text_preproc(
    series_,              # На вход получает данные в формате pandas.Series
    spec_chars_,          # список спец-символов, которые необходимо вычистить
    stopwords_            # список стоп-слов, которые необходимо убрать из текста
):          
    
    # Удаляем URL-ссылки
    series_ = series_.apply(lambda x:remove_urls(x))

    # Удаляем HTML-ссылки
    series_ = series_.apply(lambda x: remove_html(x))

    # Переводим символы к нижнему регистру
    series_ = series_.apply(lambda x: x.lower())

    # Удаляем пунктуацию и спец-символы
    series_ = series_.apply(lambda x: remove_chars_from_text(x, spec_chars_))

    # Удаляем цифры
    series_ = series_.apply(lambda x: remove_chars_from_text(x, string.digits))

    # Убираем лишние пробелы
    series_ = series_.apply(lambda x: ' '.join(x.split()))
    
    # Убираем стоп-слова
    series_ = series_.apply(lambda x: clear_stop_words(x, stopwords_))
    
    return series_


# Загрузка и обзор данных

In [3]:
# Загрузим данные
df = pd.read_csv('sample-data.csv')

# Сводная информация о загруженном датасете
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           500 non-null    int64 
 1   description  500 non-null    object
dtypes: int64(1), object(1)
memory usage: 7.9+ KB


In [4]:
# Пример данных
df.sample(10)

,id,description
293,294,Inga shortie - Short boards and short shorts l...
273,274,Torrentshell jkt - Tropical rain forests share...
94,95,Mountain island t-shirt - For those who aspire...
76,77,L/s rashguard - The 30-UPF fabric of our Long-...
209,210,Nine trails vest - Simplicity in action - this...
233,234,Vitaliti dress - The ocean has had its way wit...
154,155,Active hipster - While the perfect life might ...
49,50,Guidewater duffle - max - For serious angling ...
345,346,"Versatiliti tee - A free-moving, breathable an..."
12,13,Beach bucket - Whenever you're at the junction...


In [5]:
# Посмотрим пример текста в описании товара
df['description'][0]

'Active classic boxers - There\'s a reason why our boxers are a cult favorite - they keep their cool, especially in sticky situations. The quick-drying, lightweight underwear takes up minimal space in a travel pack. An exposed, brushed waistband offers next-to-skin softness, five-panel construction with a traditional boxer back for a classic fit, and a functional fly. Made of 3.7-oz 100% recycled polyester with moisture-wicking performance. Inseam (size M) is 4 1/2". Recyclable through the Common Threads Recycling Program.<br><br><b>Details:</b><ul> <li>"Silky Capilene 1 fabric is ultralight, breathable and quick-to-dry"</li> <li>"Exposed, brushed elastic waistband for comfort"</li> <li>5-panel construction with traditional boxer back</li> <li>"Inseam (size M) is 4 1/2"""</li></ul><br><br><b>Fabric: </b>3.7-oz 100% all-recycled polyester with Gladiodor natural odor control for the garment. Recyclable through the Common Threads Recycling Program<br><br><b>Weight: </b>99 g (3.5 oz)<br><b

Текст содержит таги форматирования и спец-символы. Перед преобразованием в векторное представление тексты необходимо от них почистить.

Далее нужно удалить цифры, перевести все символы в нижний регистр и удалить стоп-слова.

In [10]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
# Выведем стандартный перечень знаков препинания, входящий в библиотеку string
print('Базовый набор знаков препинания:', string.punctuation)

# Загрузим справочник английских стоп-слов 
stopwords_english = stopwords.words('english')

# Выведем базовый список стоп-слов
print('Базовый список стоп-слов:', stopwords_english)

# Расширим базовый список стоп-слов единицами измерения
stopwords_english.extend(['g', 'oz'])

Базовый набор знаков препинания: !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
Базовый список стоп-слов: ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',

In [12]:
# Сохраним в переменную список символов пунктуации
spec_chars = string.punctuation

# Проведем предобработку текста поля 'description'
df['description'] = text_preproc(df['description'], spec_chars, stopwords_english)

# Проверим результат предобратобки
df['description'][0]

'active classic boxers theres reason boxers cult favorite keep cool especially sticky situations quickdrying lightweight underwear takes minimal space travel pack exposed brushed waistband offers nexttoskin softness fivepanel construction traditional boxer back classic fit functional fly made recycled polyester moisturewicking performance inseam size recyclable common threads recycling program details silky capilene fabric ultralight breathable quicktodry exposed brushed elastic waistband comfort panel construction traditional boxer back inseam size fabric allrecycled polyester gladiodor natural odor control garment recyclable common threads recycling program weight made mexico'

# Векторизация текстов TFIDF

In [13]:
# Инициализируем токенайзер для преобразования каждого текста в вектор. Максимальный размер словаря 25000 слов
tfidf_vectorizer = TfidfVectorizer(binary=True, max_features=25000)

# Преобразуем очищенные тексты в матрицы TFIDF
tfidf_embedings = tfidf_vectorizer.fit_transform(df['description'])

# Проверяем размерность полученного массива
tfidf_embedings.shape

(500, 5051)

In [ ]:
# Вычисляем попарно косинусное расстояние между векторами в матрице
cos_matrix = cosine_similarity(tfidf_embedings)

# Находим индексы значений косинусного расстояния больше 0.7
pair_indexes = np.transpose(np.nonzero(cos_matrix > 0.8))

# Перебираем пары индексов
for x in pair_indexes:
    
    # Если индексы не равны друг другу, что означает, что сравнивались описания разных товаров
    if x[0] != x[1]:
        
        # Выводим пары описаний товаров
        print(df['description'][x])


# Векторизация текстов W2V

In [15]:
# Загружаем предобученную модель
word_vectors = gensim.downloader.load("word2vec-google-news-300")

[=================================================-] 99.8% 1659.8/1662.8MB downloaded


In [ ]:
# Создаем пустой DataFrame для эмбедингов текстов
docs_vectors = pd.DataFrame()

# Идем по текстам
for doc in df['description']:
    
    # Создаем пустой DataFrame для хранения векторов слов текста
    temp = pd.DataFrame()
    
    # Идем по словам в тексте
    for word in doc.split(' '):
        
        try:
            # Векторизуем слово с использованием предобученной модели W2V
            word_vec = word_vectors[word]

            # Добавляем вектор слова в DF
            temp = temp.append(pd.Series(word_vec), ignore_index = True)
            
        except:
            pass
        
    # Вычисляем вектор текста усредняя массив векторов слов
    doc_vector = temp.mean()
    
    # Добавляем вектор текста в DF эмбедингов текстов
    docs_vectors = docs_vectors.append(doc_vector, ignore_index = True)

# Размерность DF эмбедингов текстов    
docs_vectors.shape

In [17]:
# Примеры векторизованных текстов
docs_vectors.sample(5)

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
281,-0.014215,0.029400,-0.068109,0.071323,-0.076661,-0.010374,0.031660,-0.069588,0.076656,0.084322,...,-0.070450,-0.019910,-0.109974,0.031777,-0.010613,-0.034714,0.075309,-0.001497,0.024177,0.054152
311,-0.036820,0.088027,0.012051,0.081767,-0.074823,-0.003016,0.022253,-0.078171,0.074554,0.137727,...,-0.107171,-0.009779,-0.090762,-0.018926,-0.024475,-0.062446,0.069326,-0.069496,0.065305,0.011553
332,-0.041864,0.085060,-0.008363,0.093516,-0.074078,0.056390,0.054124,-0.097302,0.042922,0.106663,...,-0.094520,-0.016868,-0.052246,-0.028333,0.032556,-0.056651,0.128946,-0.077091,0.052747,-0.043324
325,-0.011082,0.023059,-0.028724,0.113606,-0.061977,-0.025908,0.051195,-0.085390,0.074606,0.115341,...,-0.096298,0.019643,-0.009331,-0.012137,0.005573,-0.043899,0.098089,-0.023705,0.037224,0.040942
155,-0.036085,0.011818,-0.076087,-0.007148,-0.067861,-0.009624,0.039668,-0.088894,0.037891,0.058915,...,-0.074969,0.065057,-0.059466,-0.015821,-0.047361,-0.013159,0.105062,0.003509,0.004253,0.034020


In [18]:
# Вычисляем попарно косинусное расстояние между векторами в матрице, округляем значения до 5 знака после запятой
cos_matrix_w2v = np.around(cosine_similarity(docs_vectors), 5)
cos_matrix_w2v

array([[1.     , 0.924  , 0.87996, ..., 0.87295, 0.87712, 0.89696],
       [0.924  , 1.     , 0.94335, ..., 0.8839 , 0.88246, 0.88969],
       [0.87996, 0.94335, 1.     , ..., 0.86466, 0.83776, 0.85104],
       ...,
       [0.87295, 0.8839 , 0.86466, ..., 1.     , 0.86754, 0.85358],
       [0.87712, 0.88246, 0.83776, ..., 0.86754, 1.     , 0.94413],
       [0.89696, 0.88969, 0.85104, ..., 0.85358, 0.94413, 1.     ]],
      dtype=float32)

In [19]:
# Находим пары векторов, косинусное расстояние между которыми больше 0.985
pair_indexes_w2v = np.transpose(np.nonzero(cos_matrix_w2v > 0.985))

# Перебираем пары индексов
for x in pair_indexes_w2v:
    
    # Если индексы не равны друг другу, что означает, что сравнивались описания разных товаров
    if x[0] != x[1]:
        
        # Выводим пары похожих описаний товаров
        print(df['description'][x])

3      alpine guide pants skin climb ice switch rock ...
158    alpine guide pants skin climb ice switch rock ...
Name: description, dtype: object
4      alpine wind jkt high ridges steep ice anything...
307    alpine wind jkt high ridges steep ice anything...
Name: description, dtype: object
7      print banded betina btm fullest coverage botto...
219    solid betina btm fullest coverage bottoms beti...
Name: description, dtype: object
14    borderless shorts go forward others cringe bor...
15    borderless shorts one summertimes gifts chance...
Name: description, dtype: object
15    borderless shorts one summertimes gifts chance...
14    borderless shorts go forward others cringe bor...
Name: description, dtype: object
19     cap graphic tshirt tee made lightestweight cap...
339    cap graphic tshirt elegant union graceful desi...
Name: description, dtype: object
24     cap bottoms sufferfests like grand january pai...
175    cap bottoms world consists granite frozen plac...
Name: de